# 0. 데이터 전처리
---
1차 목표: pdf로 작성된 자기소개서 파일 텍스트화하여 excel 파일로 저장 (220824 완료)

-> 보완사항 총 73개 항목 중 52개 데이터만 처리됨.(파일 호환 문제로 판단)

-> docx 파일 추가 예정

2차 목표: 자기소개서 질문 내용 데이터에서 배제 필요, 정규화 필요

--- 8월 28일 update ---

pdf 파일 호환성 문제로 word 파일로 변환(응용프로그램 활용)

-> word로 작성된 자기소개서 파일 텍스트화(220828 완료)

-> 학교별 양식 차이로 완벽한 정리에 시간이 걸리므로 우선 수동으로 데이터 정리 완료

-> 시간이 된다면 학교별로 분류하여 각 학교 양식 변환 방식을 반영해서 수정해볼 것

-> 자기소개서 질문 내용 데이터에서 삭제 완료

3차 목표: 학교 유형별 label 추가

In [1]:
import glob                  # 해당 디렉토리의 파일들을 탐색하기 위한 모듈
from io import StringIO      # StringIO는 파일을 흉내내는 객체로, 문자열 데이터를 임시로 처리할 때 활용
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

import re
import pandas as pd
import numpy as np

In [2]:
def read_pdf(file_path, convertStartNo = 1):
    '''파일 경로와 변환 시작 페이지를 매개변수로 하여 해당 PDF 파일을 text로 반환. 
       file_path, converStartNo = 1이 기본값'''
    output_String = StringIO()
    with open(file_path, 'rb') as file: 
        # rb는 파일을 byte 형식으로 읽어옴
        # 아래 코드들은 pdf를 byte 형식에서 다시 한글 문서로 변환하는 과정으로 추정 
        parser = PDFParser(file)                                              # 데이터를 분석하여 토큰 단위로 생성하는 듯
        doc = PDFDocument(parser)                                             # 생성된 토큰들을 다시 문서 단위로 묶는 방식으로 추정
        rsrcmgr = PDFResourceManager()                                        # pdf resource manager 객체 생성
        device = TextConverter(rsrcmgr, output_String, laparams = LAParams()) # 위의 객체를 활용하여 text 형식으로 변환?
        
        # 변환한 내역을 페이지별로 구분하는 과정으로 추정 
        interpreter = PDFPageInterpreter(rsrcmgr, device)                   
        count = 0
        for page in PDFPage.create_pages(doc):
            count += 1
            #(텍스트화를 희망하는 페이지가 아니면 skip하도록 변형)
            if count < convertStartNo: 
                continue
            else:
                interpreter.process_page(page)
        return output_String.getvalue()    

#### 불필요한 공백 제거 후 문장별로 나누기(1차 정리) : 
---
PDF to Text (저장 형식이 학교마다 다르고 글자가 깨짐)

In [3]:
import re

In [4]:
# 희망 디렉터리 안에 pdf 파일 경로 list 만들기
list_filepath = glob.glob('./Data/*.pdf')
count = 0

df = pd.DataFrame(columns = ["내용"])
# 파일 경로를 하나씩 불러와 read_pdf 함수 호출
for filepath in list_filepath:

    text = read_pdf(filepath, 2)     # 첫 페이지는 개인정보 페이지이므로 생략
    textlist = text.split(". ")
    
    text_S = str()
    for t in textlist:
        t = " ".join(t.split())
        t = re.sub(r"[^가-힣A-Za-z0-9\. ]", "", t)
        text_S = text_S + t + ". "
      
    df.loc[count] = [text_S]    
    count += 1

In [5]:
df.to_excel("./df_sample01.xlsx")

In [6]:
df_r = pd.read_excel("./df_sample01.xlsx", engine = "openpyxl", header=0, index_col=0)
df_r.head()

,내용
0,나의 꿈과 끼 인성1500자 이내 띄어쓰기 제외 접수번호 JC1022 수험번호 ...
1,나의 꿈과 끼 인성1500자띄어쓰기 제외 이내 본인이 스스로 학습계획을 세우고 ...
2,.
3,나의 꿈과 끼 인성1500자 이내 수험번호 1396 나는 세계적인 친환경 건축가로...
4,.


### 8월 28일자 word to excel

In [7]:
from docx import Document

In [8]:
def read_docx(file_path):
    doc = Document(file_path)
    text = str()
    for i, paragraph in enumerate(doc.paragraphs):
        text = text + paragraph.text
    return text

In [9]:
file_path_word = glob.glob("./Data/*.docx")
count = 0
scool=str()
df_word = pd.DataFrame(columns=["내용", "학교"])
for file_path in file_path_word:
    text = read_docx(file_path)
    text = re.sub(r"[^가-힣A-Za-z0-9\. ]", "", text)
    i = text.rfind("학교장")
    scool = text[i-10 : i-1]
    df_word.loc[count] = [text, scool]
    count += 1

In [10]:
df_word.to_excel("./df_sample02.xlsx")

#### 학교별 양식 차이로 일단 수동으로 정리 마무리 : 
---
향후 학교별 정리 알고리즘을 별도로 제작해볼 것

In [12]:
df_r2 = pd.read_excel("./df_sample03.xlsx", engine = "openpyxl", header=0, index_col=0)
df_r2.head()

,내용,학교
0,나는 공부를 할 때 반복을 중요시 하였다. 특히 수학 공부를 할 때에 오답이나 헷갈...,이화여자외국어고
1,1학년까지 문제집을 위주로 공부했지만 교과서의 세부 내용을 다룬 문제에 잘 대처하 ...,서울국제고
2,내 꿈은 영어 선생님이다 . 그래서 수업 시간에도 미래의 내가 어떻게 수업을 하...,김포외국어고
3,세계적인 건축가로서 성장하기 위해 많은 노력을 기울여 왔다. 먼저 1학년 때 3d프...,용인한국외국어대학교부설고
4,중학교 3 년 동안 영어를 단지 언어가 아닌 문화로 익 히 고 이를 친구들과 나누는...,대원외국어고


In [13]:
def make_code(school):
    if str(school).find("외국어고") != -1:
        return 1
    elif str(school).find("국제") != -1 :
        return 2
    elif str(school).find("용인") == -1 and str(school).find("하나") == -1 and str(school).find("상산") == -1 :
        return 3
    else:
        return 0

In [14]:
# 전국 자사고 0 외국어고 1 국제고 2 광역 자사고 3 코드 부여
df_r2["code"] = 0
df_r2["code"] = df_r2["code"].apply(make_code)
df_r2.head()

,내용,학교,code
0,나는 공부를 할 때 반복을 중요시 하였다. 특히 수학 공부를 할 때에 오답이나 헷갈...,이화여자외국어고,3
1,1학년까지 문제집을 위주로 공부했지만 교과서의 세부 내용을 다룬 문제에 잘 대처하 ...,서울국제고,3
2,내 꿈은 영어 선생님이다 . 그래서 수업 시간에도 미래의 내가 어떻게 수업을 하...,김포외국어고,3
3,세계적인 건축가로서 성장하기 위해 많은 노력을 기울여 왔다. 먼저 1학년 때 3d프...,용인한국외국어대학교부설고,3
4,중학교 3 년 동안 영어를 단지 언어가 아닌 문화로 익 히 고 이를 친구들과 나누는...,대원외국어고,3


In [15]:
df_r2.to_excel("./df_sample04.xlsx")